In [2]:
import numpy as np
import math 
from scipy.optimize import least_squares
from sympy import *
import glob
import cv2

In [2]:
class BundleAdjustment:
    def __init__(self, world_to_imagematrix, initRtvalues, initWorldp, imagecoords,K_intrinsic_matrix):
        self.initWorldp = initWorldp   ## no. of world points X 3 numpy array
        self.K_intrinsic_matrix = K_intrinsic_matrix
        self.world_to_imagematrix = world_to_imagematrix   ## no. of images X no. of world points. This will tell index of matched image points in respective images
        self.initRtvalues = initRtvalues  ## no.of cameras X 4 X 3
        self.imagecoords = imagecoords   ##no. of images X no. interest points with 2 or more matches in that image
        self.Rotation_to_axisangle()
        self.JacCalculator()
        self.sparsity_and_imagep_calculator()
        #initial_Jacobian = self.JacCreater(self.init_W_R_t)
    
    def Rotation_to_axisangle(self):
        Final = []    ## shape = no.of cameras X (x,y,z,theta,t1,t2,t3)

        for i in self.initRtvalues:
            m = i[:,:3]
            trans = i[:,3]
            
            epsilon = 0.01
            epsilon2 = 0.1
            if ((abs(m[0][1]-m[1][0])< epsilon) and (abs(m[0][2]-m[2][0])< epsilon) and (abs(m[1][2]-m[2][1])< epsilon)):
                # singularity found...
                # first check for identity matrix which must have +1 for all terms in leading diagonaland zero in other terms
                if ((abs(m[0][1]+m[1][0]) < epsilon2) and (abs(m[0][2]+m[2][0]) < epsilon2) and (abs(m[1][2]+m[2][1]) < epsilon2) and (abs(m[0][0]+m[1][1]+m[2][2]-3) < epsilon2)):
                    # angle= 0 degrees
                    Final.append([1,0,0,0,trans[0],trans[1],trans[2]]) 
                else:
                    # angle = 180 degrees
                    angle = np.pi;
                    xx = (m[0][0]+1)/2
                    yy = (m[1][1]+1)/2
                    zz = (m[2][2]+1)/2
                    xy = (m[0][1]+m[1][0])/4
                    xz = (m[0][2]+m[2][0])/4
                    yz = (m[1][2]+m[2][1])/4
                    
                    if ((xx > yy) and (xx > zz)):    # m[0][0] is the largest diagonal term
                        if (xx< epsilon):
                            x = 0
                            y = 0.7071
                            z = 0.7071
                        else:
                            x = math.sqrt(xx)
                            y = xy/x
                            z = xz/x

                    elif (yy > zz):    # m[1][1] is the largest diagonal term
                        if (yy< epsilon):
                            x = 0.7071
                            y = 0
                            z = 0.7071
                        else: 
                            y = math.sqrt(yy);
                            x = xy/y
                            z = yz/y
                            
                    else:    # m[2][2] is the largest diagonal term so base result on this
                        if (zz< epsilon): 
                            x = 0.7071
                            y = 0.7071
                            z = 0
                        else:
                            z = math.sqrt(zz)
                            x = xz/z
                            y = yz/z
                    Final.append([x,y,z,angle,trans[0],trans[1],trans[2]])
            else:
                # as we have reached here there are no singularities so we can handle normally
                s = math.sqrt((m[2][1] - m[1][2])*(m[2][1] - m[1][2])+(m[0][2] - m[2][0])*(m[0][2] - m[2][0])+(m[1][0] - m[0][1])*(m[1][0] - m[0][1])) # used to normalise
                if (abs(s) < 0.001):
                    s=1
                print((m[0][0] + m[1][1] + m[2][2] - 1)/2)
                angle = math.acos((m[0][0] + m[1][1] + m[2][2] - 1)/2);
                x = (m[2][1] - m[1][2])/s
                y = (m[0][2] - m[2][0])/s
                z = (m[1][0] - m[0][1])/s
                Final.append([x,y,z,angle,trans[0],trans[1],trans[2]])

        self.initR_t = np.array(Final).flatten('C')
        ## flattening world points array
        self.initWorldp = self.initWorldp.flatten('C')
        ## calculating self.init_W_R_t 
        self.init_W_R_t = np.zeros((1,self.initR_t.shape[0]+self.initWorldp.shape[0]))
        self.init_W_R_t[0,0:self.initWorldp.shape[0]] = self.initWorldp
        self.init_W_R_t[0,self.initWorldp.shape[0]:] = self.initR_t
        
    def JacCalculator(self):
        from sympy.abc import X, Y, Z, x, y, z, theta, p, q, r
        s = sin(theta)
        c = cos(theta)
        t = 1 - c
        '''R = Matrix([ [t*x*x + c, t*x*y - z*s, t*x*z + y*s],
                     [t*x*y + z*s, t*y*y + c, t*y*z - x*s],
                     [t*x*z - y*s, t*y*z + x*s, t*z*z + c] ])

        t = Matrix([ [p], [q], [r] ])'''

        P = Matrix([ [t*x*x + c, t*x*y - z*s, t*x*z + y*s, p],   ### Camera Matrix
                     [t*x*y + z*s, t*y*y + c, t*y*z - x*s, q],
                     [t*x*z - y*s, t*y*z + x*s, t*z*z + c, r] ])

        A = Matrix([ [X], [Y], [Z], [1] ])   ### World point matrix
    
        
        temp = self.K_intrinsic_matrix*P*A
        temp = temp / temp[-1] 
        temp = Matrix([temp[0], temp[1]])
        self.xhat = utilities.lambdify((X, Y, Z, x, y, z, theta, p, q, r), temp)  ### for calculating image point estimate 
        J = diff(temp, Matrix([X, Y, Z, x, y, z, theta, p, q, r ]), 1)
        self.jacfun = utilities.lambdify((X, Y, Z, x, y, z, theta, p, q, r), J)
    
    def JacCreator(self, current_matrix):  ## current matrix has (X, R|t) at current iteration and shape (1,no.)   
        ### This function calculates Jacobian at every iteration, by creating matrix of shape self.jacdims & using sympy 
        ### to fill jacobian values to be dealt with 
        current_matrix = current_matrix.reshape((1,current_matrix.shape[0]))
        num_3D = self.world_to_imagematrix.shape[1]
        
        xes, yies = np.where(self.world_to_imagematrix>=0)
        Jacobian = np.zeros((2*xes.shape[0], current_matrix.shape[1]))
        
        for i, (x, y) in enumerate(zip(xes, yies)):
            X_, Y_, Z_ = current_matrix[0][3*y : 3*y+3]
            x_, y_, z_, theta_, p_, q_, r_ = current_matrix[0][3*num_3D + 7*x : 3*num_3D + 7*x + 7]            
            
            ## now calculating temporary jacobian matrix
            tempe = self.jacfun(X_,Y_,Z_,x_,y_,z_,theta_,p_,q_,r_)
            tempe = np.asarray(tempe)
            tempe = tempe.reshape((tempe.shape[0],tempe.shape[2])).T
            
            ## now putting values of temporary jacobian into final jacobian & calculating sparse_matrix
            Jacobian[2*i : 2*i+2, 3*y : 3*y+3] = np.array([tempe[0,:3],tempe[1,:3]])
            Jacobian[2*i : 2*i+2, 3*num_3D + 7*x : 3*num_3D + 7*x+7] = np.array([tempe[0,3:10],tempe[1,3:10]])
            #done
        return Jacobian
    
    def difference_for_ls(self, current_matrix):
        ''' First argument in least squares'''
        current_matrix = current_matrix.reshape((1,current_matrix.shape[0]))
        num_3D = self.world_to_imagematrix.shape[1]
        
        xes, yies = np.where(self.world_to_imagematrix>=0)
        
        imagep_hat_ordered = []
        
        for i, (x, y) in enumerate(zip(xes, yies)):
            X_, Y_, Z_ = current_matrix[0][3*y : 3*y+3]
            x_, y_, z_, theta_, p_, q_, r_ = current_matrix[0][3*num_3D + 7*x : 3*num_3D + 7*x + 7]
            
            ## now calculating image_point_hat (i.e. estimate) 
            tempo = self.xhat(X_,Y_,Z_,x_,y_,z_,theta_,p_,q_,r_)
            imagep_hat_ordered.append([tempo[0,0],tempo[1,0]])
            
        imagep_hat_ordered = np.asarray(imagep_hat_ordered).flatten('C')
        print(abs(self.ordered_imagep - imagep_hat_ordered))
        #self.imagep_hat_ordered = imagep_hat_ordered
        return self.ordered_imagep - imagep_hat_ordered
    
    def sparsity_and_imagep_calculator(self):
        '''
           Will be called only once during initialization.  
        '''
        xes, yies = np.where(self.world_to_imagematrix>=0)
        sparse_matrix = np.zeros((2*xes.shape[0], (3*self.world_to_imagematrix.shape[1]+7*self.world_to_imagematrix.shape[0])))
        #print(sparse_matrix.shape)
        ordered_imagep = []
        
        for i, (x, y) in enumerate(zip(xes, yies)):
            ordered_imagep.append([self.imagecoords[x][self.world_to_imagematrix[x,y]][0],self.imagecoords[x][self.world_to_imagematrix[x,y]][1]])
            
            ## now calculating sparse_matrix
            sparse_matrix[(2*i):(2*i+2), (3*y):(3*y+3)] = np.ones((2,3))
            #print((3*self.world_to_imagematrix.shape[1] + 7*xes[i]),(3*self.world_to_imagematrix.shape[1] + 7*xes[i]+7))
            sparse_matrix[(2*i):(2*i+2), (3*self.world_to_imagematrix.shape[1] + 7*x):(3*self.world_to_imagematrix.shape[1] + 7*x+7)] = np.ones((2,7))
            #done
            
        self.sparse_matrix = sparse_matrix
        #print("sparse matrix - ",self.sparse_matrix)
        ordered_imagep = np.asarray(ordered_imagep).flatten('C')    
        self.ordered_imagep = ordered_imagep
        #print("ordered image - ",self.ordered_imagep)
    
    def initreturner(self):
        return self.init_W_R_t.reshape(-1)
    
    def LeastSquaresCalculator(self):
        '''
        Calulates optimal values of World co-ordinates & R & t matrices
        '''
        #print(type(self.init_W_R_t.reshape(-1)))
        result = least_squares(self.difference_for_ls, self.init_W_R_t.reshape(-1), self.JacCreator, method = "lm", jac_sparsity=self.sparse_matrix) 
        return result

In [4]:
## Function to factor Camera Matrix into R, K & t

def factor(P): 
    """ Factorize the camera matrix into K,R,t as P = K[R|t]. """ 
    R, K = np.linalg.qr(P[:,:3])
    # make diagonal of K positive 
    T = np.diag(np.sign(np.diag(K)))
    if np.linalg.det(T) < 0:
        T[1,1] = T[1,1]*(-1)
    K = np.dot(K,T) 
    R = np.dot(T,R) # T is its own inverse 
    t = np.dot(np.linalg.inv(K),P[:,3])
    return K, R, t

In [5]:
Imgs = []
for file in glob.glob('images_oxford/ho*'):
    Imgs.append(cv2.imread(file)[:,:,0])
Imgs = np.array(Imgs)


IntPnts = []
for file in glob.glob('2D/house.0*ers'):
    file = open(file, 'r')
    pnts = file.read().split('\n')[:-1]
    pnts = np.array([ pnt.split() for pnt in pnts ]).astype(float).astype(int)
    IntPnts.append(pnts)
imagecoords = np.array(IntPnts)

file = open('2D/house.nview-corners')
PntsCheck = file.read().split('\n')[:-1]
PntsCheck = np.array([ Pnt.split() for Pnt in PntsCheck ])
X, Y = np.where(PntsCheck == '*')
PntsCheck[X, Y] = '-1'
world_to_imagematrix = PntsCheck.astype(int)
world_to_imagematrix = world_to_imagematrix.T
#print(world_to_imagematrix.shape)

Camera_matrices = []
for file in glob.glob('3D/house.00*.P'):
    file = open(file, 'r')
    Pees = file.read().split('\n')[:-1]
    Pees = np.array([pnt.split() for pnt in Pees ]).astype(np.float32)
    Camera_matrices.append(Pees)
Camera_matrices = np.array(Camera_matrices)

initRtvalues = []
for cam in Camera_matrices:
    K, R, t = factor(cam)
    matr = np.zeros((3,4))
    matr[:,:3] = R
    matr[:,3] = t
    aug = np.zeros((3,4))
    aug[:,:3] = R
    aug[:,3] = t
    initRtvalues.append(aug)
initRtvalues = np.array(initRtvalues)
K_intrinsic_matrix = K
print(t/math.sqrt((t[0]*t[0])+(t[1]*t[1])+(t[2]*t[2])))

file = open('3D/house.p3d', 'r')
Worldp = file.read().split('\n')[:-1]
initWorldp = np.array([p.split() for p in Worldp ]).astype(np.float32)

[-0.9172135   0.1717847   0.35945728]


In [9]:
##Normalizing the translation coordinates
for i in initRtvalues:
    if(i[0,3]==0 and i[1,3]==0 and i[2,3]==0):
        continue
    i[:,3] = i[:,3]/(math.sqrt(i[0,3]*i[0,3]+i[1,3]*i[1,3]+i[2,3]*i[2,3]))
    
initRtvalues

array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ]],

       [[ 0.8618761 , -0.06841415, -0.50248293,  0.98145943],
        [ 0.05132826,  0.99753835, -0.04777702,  0.08134775],
        [ 0.50451462,  0.0153863 ,  0.86326604,  0.17355097]]])

In [13]:
obj = BundleAdjustment(world_to_imagematrix, initRtvalues, initWorldp, imagecoords,K_intrinsic_matrix)

0.8613402487027573


In [15]:
result = obj.LeastSquaresCalculator()

[2.64258506e+00 7.47909905e-01 5.13400246e+00 2.44398060e+00
 7.35709470e+00 1.42561735e+00 1.58376627e+01 3.14374585e+00
 3.54493076e-01 3.91030947e-01 5.88629018e+00 1.59673157e+00
 4.91088938e+00 2.25285563e+00 1.39053327e+00 4.41530940e+00
 2.23016849e+00 3.64652670e+00 3.13574473e+00 4.06439725e+00
 4.07953659e+00 5.08902081e+00 4.11798476e+00 3.97468295e+00
 5.90136071e+00 4.34311689e+00 1.68731985e+00 1.39420920e+00
 1.57956836e+00 1.13172899e+00 7.22789076e+00 2.56603042e+00
 4.69993343e-01 1.41440035e-01 1.40303792e+01 2.04861806e+00
 2.17891310e+01 5.24013280e+00 3.38128418e+01 3.86642272e+00
 1.17352287e+01 2.88203252e+00 6.39015022e+00 1.90177461e+00
 2.43172056e+00 4.52207166e+00 5.34435337e+02 6.43983941e+01
 1.32924741e+03 1.60103334e+02 7.84434047e+02 9.79095846e+01
 7.76150938e+02 9.86663761e+01 1.17248780e+03 1.23537157e+02
 7.28348751e+02 7.62868101e+01 6.92081301e+02 4.49313499e+01
 1.29363155e+03 1.91823507e+01 1.09176715e+03 2.24480132e+01
 9.95374614e+02 1.731442

In [16]:
## This function calculates error between initial and final matrices

b=result.x
(b-obj.initreturner())

array([-3.37293448e-01, -4.86422619e-01,  3.08442875e-02,  1.89013370e-02,
        5.30437441e-01, -7.15227992e-01, -2.69185711e-01,  2.61773811e-02,
        8.19280009e-01, -1.48251941e-01, -2.67667622e-01,  1.81455025e+00,
        2.39282998e-01,  1.09172109e-01, -6.72932802e-01, -8.13524398e-01,
        2.91718587e-01, -8.27760523e-01, -9.95076662e-01, -3.56377652e-02,
       -6.76716123e-01,  5.91951323e-01, -5.56060691e-01, -8.74133383e-01,
       -8.53908202e-03, -3.43120185e-01, -2.94670825e-01, -2.19617807e-01,
       -3.03293365e-01, -1.43210376e-01, -2.29943452e-01, -3.46317966e-01,
       -3.08659134e-01, -4.20301935e-01, -2.38105829e-01, -1.92081618e-01,
       -7.91069849e-01, -3.05407184e-01, -5.64190184e-01, -1.18186572e-02,
        1.20133177e-01, -2.74344012e-01, -1.27230620e-01,  1.00655589e-01,
       -9.15027965e-02, -3.88914569e-01, -2.48524162e-02,  9.18864079e-01,
       -3.35796046e-01,  1.66166326e-01,  1.75940840e-02, -2.50388550e-01,
       -9.08877928e-02,  

## Kachra

In [6]:
world_to_imagematrix = np.load("Points (2)/Point_Img_Match.npy")
world_to_imagematrix = world_to_imagematrix.T

In [7]:
K_intrinsic_matrix = np.load("Points (2)/K.npy")

In [8]:
initRtvalues = np.load("Points (2)/P1_P2.npy")

In [10]:
imagecoords = np.load("Points (2)/kp.npy")

In [11]:
initWorldp = np.load("Points (2)/3D_points.npy")

In [12]:
initWorldp.shape

(23, 3)

In [14]:
obj.world_to_imagematrix.shape

(2, 23)

### Kachra

In [253]:
a = result.x

In [254]:
a = a.reshape((a.shape[0],1))
(a-obj.initreturner())

array([[ 0.        , -2.58297992,  4.40698028, ..., -0.57168645,
        -1.66068465, -1.84835723],
       [ 2.58297992,  0.        ,  6.98996019, ...,  2.01129347,
         0.92229527,  0.73462269],
       [-4.40698028, -6.98996019,  0.        , ..., -4.97866672,
        -6.06766492, -6.25533751],
       ...,
       [ 0.57168645, -2.01129347,  4.97866672, ...,  0.        ,
        -1.0889982 , -1.27667078],
       [ 1.66068465, -0.92229527,  6.06766492, ...,  1.0889982 ,
         0.        , -0.18767259],
       [ 1.84835723, -0.73462269,  6.25533751, ...,  1.27667078,
         0.18767259,  0.        ]])

In [154]:
initRtvalues = []
for cam in Camera_matrices:
    K, R, t = factor(cam)
    matr = np.zeros((3,4))
    matr[:,:3] = R
    matr[:,3] = t
    aug = np.zeros((3,4))
    aug[:,:3] = R
    aug[:,3] = t
    initRtvalues.append(aug)
initRtvalues = np.array(initRtvalues)

In [90]:
math.acos(0)

1.5707963267948966

In [103]:
a = np.ones((7,7))
a[2:4,0:7]

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [4]:
from sympy.abc import X, Y, Z, x, y, z, theta, p, q, r
s = sin(theta)
c = cos(theta)
t = 1 - c
'''R = Matrix([ [t*x*x + c, t*x*y - z*s, t*x*z + y*s],
             [t*x*y + z*s, t*y*y + c, t*y*z - x*s],
             [t*x*z - y*s, t*y*z + x*s, t*z*z + c] ])

t = Matrix([ [p], [q], [r] ])'''

P = Matrix([ [t*x*x + c, t*x*y - z*s, t*x*z + y*s, p],   ### Camera Matrix
             [t*x*y + z*s, t*y*y + c, t*y*z - x*s, q],
             [t*x*z - y*s, t*y*z + x*s, t*z*z + c, r] ])
create_P = utilities.lambdify((x, y, z, theta), P)

In [6]:
type(create_P(1, 2, 3, 45))

numpy.ndarray